# 신경망 구현

* https://github.com/mnielsen/neural-networks-and-deep-learning/tree/master/src
* 링크 읽어보면 도움이 된다.

In [3]:
import random

class Network(object):

    def __init__(self, sizes):   #size에 number of neurons가 들어간다.
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the   #random하게 뿌려 주는 것이 좋다.
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first  
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)   #이렇게 하면 3이 들어간다.
        self.sizes = sizes              
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]   #1번째에 bias 안 걸고. 그 다음 값부터 쓰겠다. y랜덤하게 list에 넣는다.
        self.weights = [np.random.randn(y, x)                      #biases는 배열의 리스트만 된다. weights도 마찬가지. 앞단과 그 다음단 사이즈를 봐야 metrix가 된다. 
                        for x, y in zip(sizes[:-1], sizes[1:])]    #첫 번째꺼는 3X2행렬, 그 다음에는 바이어스 빼고 2X1행렬이 하나에 들어간다.

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,   #실제로 콜 할 것은 SGD다.
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)    #만약에 test_data가 있으면 
        n = len(training_data)
        for j in xrange(epochs):   #미니배치 하나 돌면 하나의 epoch가 끝난다. 예를 들어 100개 데이터, 미니배치 20개면 웨이트는 업데이트 한다.
            # 미니배치 이터레이션 다 끝나면 epoch 1번 끝난거. 데이터 만개고 미니배치 100개면 1000번 돌고 1번의 epoch가 끝난 것.
            random.shuffle(training_data)   #랜덤셔플 하는 이유는 batch를 뽑기 위해 랜덤하게 뽑으려고
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)   #업데이트 하는 거면 weight가 업데이트 된다.
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))   #이밸류에이트로 이터레이터를 돈다.
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):   #이게 실제로 하는 부분
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]  
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)   #델타 값이 그레디언트 값이다. 백프로프는 실제 계산해주는 함수. 한 포인트에 대해서. xi, yi에 대해서
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] 
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw   #nw가 그레디언트 값이다.
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):   #이게 실제 계산하는 것
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]   #공란을 만들어 놓는다.
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x    #여기서. 닐슨은 비샵이랑 기호 반대로. a가 z, z를 a로. 그래서 여기서 말하는 액티베이션은 z. x는 a값이다.
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):   #foward 과정이다.
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass   #이건 최종 델타다. 이건 오차다. cost_derivative. MSE를 썼는데
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])   #항상 미분값을 곱해줘야 한다. 이러면 최초단의 델타값이 된다.
        nabla_b[-1] = delta   #내일 증명식을 보여주면서 다시. 여기서 dot은 메트릭스 결과값이 나오게끔
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())   
        return (nabla_b, nabla_w)   #그래서업데이트로 돌아간다.

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)   #셀프 포워드 하면 마지막 나온 y값. 제일 큰 번호를 뽑는다. 그리고 y값과 비교
                        for (x, y) in test_data]       
        return sum(int(x == y) for (x, y) in test_results)   #여기서의 x는 y햇 값이다. true, 1의 값을 sum해준다. 이게 accuracy가 된다.

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):   #시그모이드를 실제로 손으로 계산한 값이다.
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

## MNIST Digit Image Test

In [4]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split

mnist = fetch_mldata('MNIST original')
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target)
X_train_new = [np.reshape(x, (784, 1)) for x in X_train]
y_train_new = [y[:, np.newaxis] for y in OneHotEncoder().fit_transform(y_train[:, np.newaxis]).toarray()]
X_test_new = [np.reshape(x, (784, 1)) for x in X_test]
training_data = zip(X_train_new, y_train_new)
test_data = zip(X_test_new, y_test)

## 패키지 사용방법

In [5]:
%cd /home/dockeruser/neural-networks-and-deep-learning/src
%ls

/home/dockeruser/neural-networks-and-deep-learning/src
conv.py                    mnist_loader.py   network2.py   network.py
expand_mnist.py            mnist_loader.pyc  network2.pyc  network.pyc
mnist_average_darkness.py  mnist_svm.py      network3.py   old/


In [6]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [7]:
import network

net = network.Network([784, 30, 10])
%time net.SGD(training_data, 30, 10, 0.1, test_data=test_data)   #30번 이터레이션 돈다. 러닝에잇는 에타. 0.1. 스텝 사이즈.

Epoch 0: 4291 / 10000
Epoch 1: 6391 / 10000
Epoch 2: 7183 / 10000
Epoch 3: 7581 / 10000
Epoch 4: 7800 / 10000
Epoch 5: 7905 / 10000
Epoch 6: 8031 / 10000
Epoch 7: 8570 / 10000
Epoch 8: 8728 / 10000
Epoch 9: 8797 / 10000
Epoch 10: 8874 / 10000
Epoch 11: 8936 / 10000
Epoch 12: 8963 / 10000
Epoch 13: 9000 / 10000
Epoch 14: 9018 / 10000
Epoch 15: 9061 / 10000
Epoch 16: 9070 / 10000
Epoch 17: 9095 / 10000
Epoch 18: 9106 / 10000
Epoch 19: 9122 / 10000
Epoch 20: 9125 / 10000
Epoch 21: 9147 / 10000
Epoch 22: 9166 / 10000
Epoch 23: 9169 / 10000
Epoch 24: 9183 / 10000
Epoch 25: 9192 / 10000
Epoch 26: 9203 / 10000
Epoch 27: 9205 / 10000
Epoch 28: 9227 / 10000
Epoch 29: 9228 / 10000
CPU times: user 10min 47s, sys: 250 ms, total: 10min 47s
Wall time: 10min 57s


xrange는 속도 때문에 쓴다. range는 진짜로 변수를 만들어서 채워 넣는다. range 100만이면 진짜 만들어서 이터레이션 돌린다. xrange는 만들지 않고 이터레이터만 돌리기 때문에 빠르다. 그냥 이터레이터다.